In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
import os
from datetime import datetime
import re
import hashlib
import random
import requests
from bs4 import BeautifulSoup
import json

def setup_driver_advanced():
    """高级浏览器设置 - 绕过检测"""
    chrome_options = Options()
    
    # 基础设置
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    
    # 高级反检测设置
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation", "enable-logging"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # 禁用图片和CSS加速加载
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-plugins")
    chrome_options.add_argument("--disable-javascript")
    
    # 随机用户代理
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    ]
    selected_ua = random.choice(user_agents)
    chrome_options.add_argument(f"--user-agent={selected_ua}")
    
    driver = webdriver.Chrome(options=chrome_options)
    
    # 执行高级反检测脚本
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3, 4, 5]})")
    driver.execute_script("Object.defineProperty(navigator, 'languages', {get: () => ['zh-CN', 'zh', 'en']})")
    
    # 设置超时
    driver.set_page_load_timeout(30)
    driver.set_script_timeout(30)
    
    return driver

def scrape_carousell_advanced():
    """高级爬虫 - 突破500条限制"""
    driver = None
    all_products = []
    seen_products = set()
    
    try:
        driver = setup_driver_advanced()
        
        # 策略1: 使用不同的搜索URL和参数
        search_urls = [
            "https://www.carousell.com.hk/search/dimoo",
            "https://www.carousell.com.hk/search/dimoo?sort_by=time_created%2Cdescending",
            "https://www.carousell.com.hk/search/dimoo?sort_by=price%2Cascending",
            "https://www.carousell.com.hk/search/dimoo?addRecent=false"
        ]
        
        for search_url in search_urls:
            if len(all_products) >= 1000:  # 如果已经收集足够多数据
                break
                
            print(f"\n🎯 尝试URL: {search_url}")
            products_from_url = scrape_with_url(driver, search_url, seen_products)
            all_products.extend(products_from_url)
            print(f"从该URL获得 {len(products_from_url)} 个商品")
            
            # 休息一下避免被封
            time.sleep(random.uniform(5, 10))
        
        print(f"\n总共收集到 {len(all_products)} 个唯一商品")
        
        if all_products:
            saved_file = save_advanced_data(all_products)
            return all_products, saved_file
        else:
            return None, None

    except Exception as e:
        print(f"高级爬取过程中出错: {e}")
        import traceback
        traceback.print_exc()
        
        if all_products:
            saved_file = save_advanced_data(all_products)
            return all_products, saved_file
        return None, None
    finally:
        if driver:
            driver.quit()

def scrape_with_url(driver, search_url, seen_products):
    """使用特定URL爬取"""
    products = []
    
    try:
        print(f"访问: {search_url}")
        driver.get(search_url)
        
        # 等待页面加载
        time.sleep(5)
        
        # 模拟人类行为滚动
        human_like_scroll(driver)
        
        max_pages = 20  # 每URL最多20页
        current_page = 1
        
        while current_page <= max_pages:
            print(f"处理第 {current_page} 页...")
            
            # 深度滚动加载更多内容
            deep_scroll_for_content(driver)
            
            # 提取当前页商品
            page_products = extract_products_advanced(driver, seen_products)
            products.extend(page_products)
            
            print(f"第 {current_page} 页找到 {len(page_products)} 个新商品")
            
            # 检查是否还有更多页面
            if not has_more_content(driver):
                print("没有更多内容，停止翻页")
                break
                
            # 尝试翻页
            if not smart_pagination(driver, current_page):
                print("无法翻页，停止")
                break
                
            current_page += 1
            
            # 随机延迟，模拟人类行为
            time.sleep(random.uniform(3, 8))
            
            # 每5页休息更长时间
            if current_page % 5 == 0:
                print("休息一下...")
                time.sleep(random.uniform(10, 15))
    
    except Exception as e:
        print(f"该URL爬取失败: {e}")
    
    return products

def human_like_scroll(driver):
    """模拟人类滚动行为"""
    print("模拟人类滚动...")
    
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    viewport_height = driver.execute_script("return window.innerHeight")
    
    # 分段滚动，带有随机停顿
    scroll_positions = [
        viewport_height * 0.3,
        viewport_height * 0.6,
        viewport_height * 0.9,
        scroll_height * 0.2,
        scroll_height * 0.4,
        scroll_height * 0.6,
        scroll_height * 0.8,
        scroll_height
    ]
    
    for i, position in enumerate(scroll_positions):
        # 随机滚动速度
        scroll_script = f"""
        window.scrollTo({{
            top: {position},
            behavior: 'smooth'
        }});
        """
        driver.execute_script(scroll_script)
        
        # 随机等待时间
        wait_time = random.uniform(0.5, 2.5)
        time.sleep(wait_time)
        
        # 偶尔随机小范围滚动
        if random.random() < 0.3:
            small_scroll = random.randint(-100, 100)
            driver.execute_script(f"window.scrollBy(0, {small_scroll})")
            time.sleep(0.5)

def deep_scroll_for_content(driver):
    """深度滚动以加载更多内容"""
    print("深度滚动加载内容...")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    no_change_count = 0
    
    for i in range(10):  # 最多尝试10次
        # 滚动到底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # 检查是否有"Load More"按钮并点击
        if click_load_more_buttons(driver):
            time.sleep(3)
        
        # 检查高度是否变化
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            no_change_count += 1
            if no_change_count >= 3:
                print("连续3次滚动高度无变化，停止滚动")
                break
        else:
            no_change_count = 0
            print(f"发现新内容，页面高度: {new_height}")
        
        last_height = new_height
        
        # 随机等待
        time.sleep(random.uniform(1, 3))

def click_load_more_buttons(driver):
    """点击所有可能的加载更多按钮"""
    load_more_selectors = [
        "button[data-testid*='load-more']",
        "button:contains('Load More')",
        "button:contains('顯示更多')",
        "button:contains('Show More')",
        ".load-more",
        "[class*='load-more']"
    ]
    
    for selector in load_more_selectors:
        try:
            # 使用XPath包含文本的查找
            buttons = driver.find_elements(By.XPATH, 
                f"//button[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(text(), '顯示更多')]")
            
            for button in buttons:
                try:
                    if button.is_displayed() and button.is_enabled():
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", button)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", button)
                        print("✅ 点击了加载更多按钮")
                        return True
                except:
                    continue
        except:
            continue
    
    return False

def extract_products_advanced(driver, seen_products):
    """高级商品提取"""
    products = []
    
    # 多种元素定位策略
    extraction_strategies = [
        extract_by_data_testid,
        extract_by_class_patterns,
        extract_by_link_analysis,
        extract_by_structural_analysis
    ]
    
    all_found_products = []
    
    for strategy in extraction_strategies:
        try:
            found_products = strategy(driver, seen_products)
            if found_products:
                all_found_products.extend(found_products)
                print(f"策略 {strategy.__name__} 找到 {len(found_products)} 个商品")
                
                # 如果找到很多商品，可以提前停止
                if len(all_found_products) > 50:
                    break
        except Exception as e:
            print(f"策略 {strategy.__name__} 失败: {e}")
            continue
    
    # 去重并添加到结果
    for product in all_found_products:
        product_id = create_product_id(product)
        if product_id not in seen_products:
            seen_products.add(product_id)
            products.append(product)
    
    return products

def extract_by_data_testid(driver, seen_products):
    """通过data-testid属性提取"""
    products = []
    
    testid_patterns = [
        'listing-card',
        'product-card',
        'listing',
        'product',
        'item'
    ]
    
    for pattern in testid_patterns:
        selector = f'[data-testid*="{pattern}"]'
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                product = extract_from_element(element)
                if product and product not in products:
                    products.append(product)
        except:
            continue
    
    return products

def extract_by_class_patterns(driver, seen_products):
    """通过类名模式提取"""
    products = []
    
    class_patterns = [
        'listingCard',
        'productCard',
        'listing',
        'card',
        'item'
    ]
    
    for pattern in class_patterns:
        selectors = [
            f'div[class*="{pattern}"]',
            f'article[class*="{pattern}"]',
            f'section[class*="{pattern}"]',
            f'a[class*="{pattern}"]'
        ]
        
        for selector in selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    product = extract_from_element(element)
                    if product and product not in products:
                        products.append(product)
            except:
                continue
    
    return products

def extract_by_link_analysis(driver, seen_products):
    """通过链接分析提取"""
    products = []
    
    try:
        # 查找所有商品链接
        link_elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/p/"]')
        
        for link_element in link_elements:
            try:
                # 获取链接周围的容器
                container = link_element
                for i in range(3):
                    try:
                        container = container.find_element(By.XPATH, "./..")
                        if container.text and len(container.text.strip()) > 30:
                            product = extract_from_element(container)
                            if product and product not in products:
                                products.append(product)
                            break
                    except:
                        break
            except:
                continue
                
    except Exception as e:
        print(f"链接分析失败: {e}")
    
    return products

def extract_by_structural_analysis(driver, seen_products):
    """通过结构分析提取"""
    products = []
    
    try:
        # 查找包含商品信息的结构块
        structural_selectors = [
            'div > div > div',  # 三层div结构
            'article > div',
            'section > div',
            '[role="listitem"]'
        ]
        
        for selector in structural_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    text = element.text.strip()
                    # 检查是否包含商品特征
                    if (len(text) > 50 and 
                        ('dimoo' in text.lower() or 'HK$' in text or '$' in text)):
                        product = extract_from_element(element)
                        if product and product not in products:
                            products.append(product)
            except:
                continue
                
    except Exception as e:
        print(f"结构分析失败: {e}")
    
    return products

def extract_from_element(element):
    """从元素提取商品信息"""
    try:
        text = element.text.strip()
        if len(text) < 30:
            return None
        
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        if len(lines) < 3:
            return None
        
        # 提取信息
        title = find_title(lines)
        price = find_price(lines)
        seller = find_seller(lines)
        time_text = find_time(lines)
        link = find_link(element)
        
        if not title:
            return None
            
        return {
            'title': clean_text(title),
            'price': price or '價格待詢',
            'seller': clean_text(seller) if seller else '',
            'post_time': clean_text(time_text) if time_text else '',
            'link': link,
            'crawl_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
    except Exception as e:
        return None

def find_title(lines):
    """查找标题"""
    # 优先查找包含dimoo的行
    for line in lines:
        if 'dimoo' in line.lower() and 10 < len(line) < 100:
            return line
    
    # 返回最长的合适行
    candidate_lines = [line for line in lines if 20 < len(line) < 150]
    if candidate_lines:
        return max(candidate_lines, key=len)
    
    return lines[0] if lines else ""

def find_price(lines):
    """查找价格"""
    for line in lines:
        price_match = re.search(r'(HK\$|\$)\s*(\d+(?:,\d+)*(?:\.\d+)?)', line)
        if price_match:
            return f"{price_match.group(1)}{price_match.group(2)}"
    
    for line in lines:
        if 'free' in line.lower() or '免費' in line:
            return "FREE"
    
    return ""

def find_seller(lines):
    """查找卖家"""
    if len(lines) > 0:
        return lines[0]
    return ""

def find_time(lines):
    """查找时间"""
    time_patterns = [
        r'\d+\s*(分鐘|分钟|小時|小时|天|日|月|年)\s*前',
        r'\d+\s*(minute|hour|day|week|month|year)s?\s*ago',
        r'剛剛|剛才|just now'
    ]
    
    for line in lines:
        for pattern in time_patterns:
            if re.search(pattern, line, re.IGNORECASE):
                return line
    return ""

def find_link(element):
    """查找链接"""
    try:
        if element.tag_name == "a":
            return element.get_attribute("href")
        else:
            link_elem = element.find_element(By.CSS_SELECTOR, 'a[href*="/p/"]')
            return link_elem.get_attribute("href") if link_elem else ""
    except:
        return ""

def clean_text(text):
    """清理文本"""
    return re.sub(r'\s+', ' ', text).strip()

def has_more_content(driver):
    """检查是否还有更多内容"""
    try:
        # 检查加载更多按钮
        load_more_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(text(), '顯示更多')]"
        )
        if load_more_buttons:
            return True
        
        # 检查下一页按钮
        next_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Next') or contains(text(), '下一頁')]"
        )
        if next_buttons:
            return True
            
        # 检查是否到达底部
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        client_height = driver.execute_script("return document.documentElement.clientHeight")
        scroll_top = driver.execute_script("return document.documentElement.scrollTop")
        
        if scroll_top + client_height < scroll_height - 100:
            return True
            
    except:
        pass
    
    return False

def smart_pagination(driver, current_page):
    """智能翻页"""
    try:
        # 方法1: 点击下一页按钮
        next_buttons = driver.find_elements(By.XPATH,
            "//button[contains(text(), 'Next') or contains(text(), '下一頁')]"
        )
        
        for button in next_buttons:
            try:
                if button.is_displayed() and button.is_enabled():
                    driver.execute_script("arguments[0].click();", button)
                    print("✅ 点击下一页按钮")
                    time.sleep(3)
                    return True
            except:
                continue
        
        # 方法2: URL翻页
        next_url = f"https://www.carousell.com.hk/search/dimoo?page={current_page + 1}"
        try:
            driver.get(next_url)
            print(f"✅ 跳转到第 {current_page + 1} 页")
            time.sleep(3)
            return True
        except:
            pass
            
        # 方法3: 滚动加载
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        return True
        
    except Exception as e:
        print(f"翻页失败: {e}")
        return False

def create_product_id(product_info):
    """创建商品唯一标识"""
    key_string = f"{product_info['title']}_{product_info['price']}_{product_info.get('link', '')}"
    return hashlib.md5(key_string.encode('utf-8')).hexdigest()

def save_advanced_data(products):
    """保存高级数据"""
    if not products:
        return None
        
    try:
        desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
        filename = f"carousell_dimoo_advanced_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
        filepath = os.path.join(desktop_path, filename)
        
        df = pd.DataFrame(products)
        
        # 列映射
        column_mapping = {
            'title': '商品名称',
            'price': '价格',
            'seller': '发布用户', 
            'post_time': '发布时间',
            'link': '商品链接',
            'crawl_time': '爬取时间'
        }
        
        df = df.rename(columns=column_mapping)
        available_columns = [col for col in column_mapping.values() if col in df.columns]
        df = df[available_columns]
        
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"✅ 高级数据已保存: {filepath}")
        print(f"📊 总数据量: {len(df)} 条记录")
        
        return filepath
        
    except Exception as e:
        print(f"保存失败: {e}")
        return None

def run_mass_scraper():
    """运行大规模爬取"""
    print("🚀 开始大规模爬取Carousell Molly商品...")
    print("🎯 目标: 突破500条限制，获取1000+条数据")
    
    all_products = []
    max_attempts = 3
    
    for attempt in range(max_attempts):
        print(f"\n{'='*60}")
        print(f"尝试 {attempt + 1}/{max_attempts}")
        print(f"{'='*60}")
        
        products, saved_file = scrape_carousell_advanced()
        
        if products:
            all_products.extend(products)
            print(f"✅ 本次获取 {len(products)} 条数据")
            print(f"📊 累计获取 {len(all_products)} 条数据")
            
            if len(all_products) >= 800:
                print("🎉 已达到目标数据量!")
                break
        else:
            print("❌ 本次爬取失败")
        
        # 长时间休息避免被封
        if attempt < max_attempts - 1:
            wait_time = random.randint(30, 60)
            print(f"⏰ 等待 {wait_time} 秒后继续...")
            time.sleep(wait_time)
    
    # 最终保存
    if all_products:
        final_file = save_advanced_data(all_products)
        print(f"\n🎉 最终结果: 成功获取 {len(all_products)} 条数据!")
        print(f"💾 保存到: {final_file}")
        return all_products, final_file
    else:
        print("❌ 所有尝试都失败了")
        return None, None

if __name__ == "__main__":
    # 运行大规模爬取
    products, saved_file = run_mass_scraper()